<a href="https://colab.research.google.com/github/AashiDutt/Data-Pipelines-with-TensorFlow-Data-Services/blob/master/TFDataServices_Legacy_vs_S3_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
    %tensorflow_version 2.x
except:
    pass

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.2.0-rc3


**Merging Splits with the Legacy API**

In the Legacy API, you can merge splits together by adding them together, as shown below:

In [3]:

ds = tfds.load("mnist", split='train+test')

print("Number of Records: {:,}".format(len(list(ds))))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.
Number of Records: 70,000


***Subsplitting with the Legacy API***

With the Legacy API, we can use the subsplit method to divide the datasets. In the example below, we divide the training set into four splits by specifying the number of subsplits, with the argument k=4.

## ***NEEDS UPDATION***

In [0]:
# s1, s2, s3, s4 = tfds.Split.TRAIN.subsplit(k=4)

# dataset_split_1 = tfds.load("mnist", split=s1)
# dataset_split_2 = tfds.load("mnist", split=s2)
# dataset_split_3 = tfds.load("mnist", split=s3)
# dataset_split_4 = tfds.load("mnist", split=s4)

# print(len(list(dataset_split_1)))
# print(len(list(dataset_split_2)))
# print(len(list(dataset_split_3)))
# print(len(list(dataset_split_4)))

We can also perform the same operation, by specifying a percentage slice in the subsplit method instead. In the example below, we divide the training set into four splits by specifying a percentage slice, with tfds.percent.

In [0]:
# s1 = tfds.Split.TRAIN.subsplit(tfds.percent[0:25])
# s2 = tfds.Split.TRAIN.subsplit(tfds.percent[25:50])
# s3 = tfds.Split.TRAIN.subsplit(tfds.percent[50:75])
# s4 = tfds.Split.TRAIN.subsplit(tfds.percent[75:100])

# dataset_split_1 = tfds.load("mnist", split=s1)
# dataset_split_2 = tfds.load("mnist", split=s2)
# dataset_split_3 = tfds.load("mnist", split=s3)
# dataset_split_4 = tfds.load("mnist", split=s4)

# print(len(list(dataset_split_1)))
# print(len(list(dataset_split_2)))
# print(len(list(dataset_split_3)))
# print(len(list(dataset_split_4)))

# ***Using the New S3 API***

Before using the new S3 API, we must first find out whether the MNIST dataset implements the new S3 API. In the cell below we indicate that we want to use version 3.*.* of the MNIST dataset.

S - Split

S - Slices

S - Strings

In [8]:
mnist_builder = tfds.builder("mnist:3.*.*")

print(mnist_builder.version.implements(tfds.core.Experiment.S3))

True


We can see that the code above printed True, which means that version 3.*.* of the MNIST dataset supports the new S3 API.

Now, let's see how we can use the S3 API to download the MNIST dataset and specify the splits we want use. In the code below we download the train and test splits of the MNIST dataset and then we print their size. We will see that there are 60,000 records in the training set and 10,000 in the test set.

In [9]:
train_ds, test_ds = tfds.load('mnist:3.*.*', split=['train', 'test'])

print(len(list(train_ds)))
print(len(list(test_ds)))

60000
10000


In the S3 API we can use strings to specify the slicing instructions. For example, in the cell below we will merge the training and test sets by passing the string ’train+test' to the split argument.

In [10]:
combined = tfds.load('mnist:3.*.*', split='train+test')

print(len(list(combined)))

70000


We can also use Python style list slicers to specify the data we want. For example, we can specify that we want to take the first 10,000 records of the train split with the string 'train[:10000]', as shown below:

In [11]:
first10k = tfds.load('mnist:3.*.*', split='train[:10000]')

print(len(list(first10k)))

10000


The S3 API, also allows us to specify the percentage of the data we want to use. For example, we can select the first 20% of the training set with the string 'train[:20%]', as shown below:

In [12]:
first20p = tfds.load('mnist:3.*.*', split='train[:20%]')

print(len(list(first20p)))

12000


We can see that first20p contains 12,000 records, which is indeed 20% the total number of records in the training set. Recall that the training set contains 60,000 records.

Because the slices are string-based we can use loops, like the ones shown below, to slice up the dataset and make some pretty complex splits. For example, the loops below create 10 complimentary validation and training sets (each loop returns a list with 5 data sets).

In [0]:
val_ds = tfds.load('mnist:3.*.*', split=['train[{}%:{}%]'.format(k, k+20) for k in range(0, 100, 20)])

train_ds = tfds.load('mnist:3.*.*', split=['train[:{}%]+train[{}%:]'.format(k, k+20) for k in range(0, 100, 20)])

In [14]:
val_ds

[<DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>]

In [15]:
train_ds

[<DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>]

In [16]:
print(len(list(val_ds)))
print(len(list(train_ds)))

5
5


The S3 API also allows us to compose new datasets by using pieces from different splits. For example, we can create a new dataset from the first 10% of the test set and the last 80% of the training set, as shown below.

In [17]:
composed_ds = tfds.load('mnist:3.*.*', split='test[:10%]+train[-80%:]')

print(len(list(composed_ds)))

49000
